In [13]:

# !pip install yfinance
import unicodedata
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import math


In [14]:
sharekhan_portfolio_df = pd.read_csv("sharekhan.csv",skiprows=4)
sharekhan_stocks_list=sharekhan_portfolio_df['Stock Name'].to_list()
start_date ="2023-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

In [15]:
icici_portfolio_df = pd.read_csv("icici_direct.csv")
icici_stocks_list = icici_portfolio_df['Stock Symbol'].to_list()
# icici_stocks_list
# len(icici_stocks_list)
# icici_portfolio_df.columns
# sharekhan_stocks_list

In [16]:

sharekhan_stocks_list = list(filter(lambda x: not isinstance(x, float) or not math.isnan(x),sharekhan_stocks_list))
# icici_stocks_list


In [17]:
df = pd.read_csv('EQUITY.csv')
df=df[df[' ISIN NUMBER'].isin(icici_portfolio_df['ISIN Code'].to_list())]
stocks_list = df['SYMBOL'].tolist()
stocks_list_NSE = list(map(lambda x: x+".NS",stocks_list))
stocks_list_Sharekhan = list(map(lambda x: x+".NS",sharekhan_stocks_list))
final_list = [ls.replace(u'\xa0', u'')  for ls in stocks_list_Sharekhan]
final_list = final_list + stocks_list_NSE
final_list = list(dict.fromkeys(final_list))
print(f" Final List = {final_list}")
yahoo_data = yf.download(final_list,start=start_date,end=end_date)

 Final List = ['ASIANPAINT.NS', 'BEL.NS', 'BHARTIARTL.NS', 'DABUR.NS', 'GODREJCP.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS', 'INDUSINDBK.NS', 'ITC.NS', 'JUBLFOOD.NS', 'KOTAKBANK.NS', 'LT.NS', 'MARUTI.NS', 'PIDILITIND.NS', 'PIIND.NS', 'RELIANCE.NS', 'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'WIPRO.NS', 'ABB.NS', 'ADANIGREEN.NS', 'AXISBANK.NS', 'BAJAJFINSV.NS', 'COALINDIA.NS', 'CONCORDBIO.NS', 'CYIENT.NS', 'DRREDDY.NS', 'ENGINERSIN.NS', 'GNFC.NS', 'HAVELLS.NS', 'INFY.NS', 'JIOFIN.NS', 'KAJARIACER.NS', 'KEC.NS', 'KNRCON.NS', 'LICI.NS', 'LTTS.NS', 'NCC.NS', 'NEULANDLAB.NS', 'PAYTM.NS', 'PNCINFRA.NS', 'RVNL.NS', 'SBILIFE.NS', 'SBIN.NS', 'TITAGARH.NS', 'TRENT.NS', 'TV18BRDCST.NS', 'YESBANK.NS']
[*********************100%***********************]  50 of 50 completed


In [18]:
# for s in final_list:
#     print(yahoo_data[('Adj Close',s)])


# yahoo_data.loc[:,('Volume','ABB.NS')]
# yahoo_data.to_csv("yahoo_data.csv")
yahoo_data
data_single_index=yahoo_data.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index(level=1)
# yahoo_data.stack(level=1).rename_axis(['Date', 'Ticker'])

In [19]:
data_single_index.to_csv("yahoo.csv")

In [20]:
import warnings
warnings.filterwarnings('ignore')

sell_df=pd.DataFrame()
buy_df = pd.DataFrame()

for ticker in final_list:
    temp_df = data_single_index[data_single_index['Ticker']==ticker]
    # temp_df['rolling_volume'] = temp_df['Volume'].rolling(window='30D').mean()
    temp_df.loc[:, 'rolling_volume'] = temp_df['Volume'].rolling(window='30D').mean()
    temp_df['rolling_volume']=temp_df['rolling_volume'].astype(int)
    temp_df['Volume']=temp_df['Volume'].astype(int)
    # temp_df['greater_than_rolling_mean'] = temp_df.groupby('Ticker')['Volume'].rolling(window=3).apply(lambda x: (x > x.mean()).all()).reset_index(0, drop=True)
    temp_df['vol_gt_avg']=temp_df.apply(lambda row:1 if row['Volume']> row['rolling_volume'] else 0 , axis=1)
    last_three_days = temp_df.iloc[-4:]
    price_increase_in_last_three_days = last_three_days['Close'].diff().dropna() > 0
    price_decrease_in_last_three_days = last_three_days['Close'].diff().dropna() < 0
    volume_flag = last_three_days['vol_gt_avg'].all()==1
#     print("for ticker ", ticker)
#     print(volume_flag)
    if price_decrease_in_last_three_days.all() and volume_flag:
#         print(f' Sell Signal for : {ticker}')
#         print(last_three_days)
        sell_df = pd.concat([sell_df,last_three_days])
    if price_increase_in_last_three_days.all() and volume_flag:
#         print(f' Buy Signal for : {ticker}' )
#         print(last_three_days)
        buy_df=pd.concat([buy_df,last_three_days])
    

print(f'BUY recommendation are {buy_df[["Ticker","Close"]]}' )
print(f'SELL recommendation are {sell_df[["Ticker","Close"]]}')

BUY recommendation are              Ticker        Close
Date                            
2024-02-02  LICI.NS   945.450012
2024-02-05  LICI.NS   998.849976
2024-02-06  LICI.NS  1025.849976
2024-02-07  LICI.NS  1044.449951
SELL recommendation are                  Ticker       Close
Date                               
2024-02-02  JUBLFOOD.NS  494.850006
2024-02-05  JUBLFOOD.NS  484.850006
2024-02-06  JUBLFOOD.NS  479.850006
2024-02-07  JUBLFOOD.NS  476.000000


In [21]:
buy_df[['Ticker','Close']]

,Ticker,Close
Date,,
2024-02-02,LICI.NS,945.450012
2024-02-05,LICI.NS,998.849976
2024-02-06,LICI.NS,1025.849976
2024-02-07,LICI.NS,1044.449951


In [22]:
price_increase_in_last_three_days.all()

False

In [23]:
last_three_days['vol_gt_avg'].all()

False

In [24]:
buy_df[['Close','Ticker']]

,Close,Ticker
Date,,
2024-02-02,945.450012,LICI.NS
2024-02-05,998.849976,LICI.NS
2024-02-06,1025.849976,LICI.NS
2024-02-07,1044.449951,LICI.NS
